In [1]:
import preprocess as pre
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, TimeDistributed
from tensorflow.keras.layers import Concatenate, Flatten
from tensorflow.keras.layers import GRU, Conv2D, MaxPooling2D, Embedding
from tensorflow.keras.layers import Input, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow.keras as keras

In [2]:
from data_gen import *
from models import *

In [3]:
char2int, feat2val, max_r, max_w = pre.process()
data = pre.convert(char2int, feat2val, max_r, max_w)

In [4]:
int2char = {val: key for val, key in enumerate(char2int)}

In [5]:
batch_size = 128

In [6]:
gen = pre.gen(data, batch_size)

In [7]:
max_root = max_r + 2
max_word = max_w + 2
n_feature = data[1].shape[1]
hidden_size = 256
feat_embed_size = 32
char_embed_size = 32
n_batches = len(data[0]) // batch_size
print("Total Data: {0} Total Batches {1}".format(len(data[0]), n_batches))

Total Data: 12595 Total Batches 98


In [8]:
def embed_model(vocab_size, input_length, decoder_input, dec_output_length, 
                    n_feature, hidden_size, feat_units = 15, embed_size=64):
    root_word_input = Input(shape=(input_length, ), name="root_word_input")
    feature_input = Input(shape=(n_feature,), name="word_feature_input")

    feat_out = Dense(feat_units, activation="relu", name="feature_embedding")(feature_input)
    embedding = Embedding(vocab_size, embed_size, input_length=input_length, name="char_embedding")
    x = embedding(root_word_input)
    
    x = Reshape([input_length, embed_size, 1])(x)
#     x = keras.backend.expand_dims(x, -1)
    x = Conv2D(32, (3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D(2, 2)(x)

    x = Flatten()(x)
    x = Concatenate()([x, feat_out])
    state_h = Dense(hidden_size, activation='relu')(x)
    state_c = Dense(hidden_size, activation='relu')(x)
    state = [state_h, state_c]
    
    decoder_inputs = Input(shape=(None, ), name="target_word_input")
    emb_dec = embedding(decoder_inputs)
    
    decoder_lstm_1 = LSTM(hidden_size, return_sequences=True, return_state=True, name="decoder_gru")
    decoder_output_1, _, _= decoder_lstm_1(emb_dec, initial_state=state)
    
    decoder_lstm_2 = LSTM(hidden_size, return_sequences=True, return_state=True, name="layer2lstm")
    decoder_output_2, _, _ = decoder_lstm_2(decoder_output_1, initial_state=state)
    
    decoder_dense = Dense(dec_output_length, activation='softmax', name="train_output")
    decoder_outputs = decoder_dense(decoder_output_2)
    
    model = Model([root_word_input, feature_input, decoder_inputs], decoder_outputs)
    encoder_model = Model([root_word_input, feature_input], state)
    
    decoder_state_input_1 = [Input(shape=(hidden_size,)), Input(shape=(hidden_size,))]
    decoder_state_input_2 = [Input(shape=(hidden_size,)), Input(shape=(hidden_size,))]
    decoder_state_input = decoder_state_input_1 + decoder_state_input_2
    
    decoder_output_1, state_h_1, state_c_1 = decoder_lstm_1(emb_dec, initial_state=decoder_state_input_1)
    decoder_output_2, state_h_2, state_c_2 = decoder_lstm_2(decoder_output_1, initial_state=decoder_state_input_2)

    states = [state_h_1, state_c_1, state_h_1, state_c_1]
    decoder_outputs = decoder_dense(decoder_output_2)
    decoder_model = Model([decoder_inputs] + decoder_state_input, [decoder_outputs] + states)

    return model, encoder_model, decoder_model

In [9]:
model, encoder, decoder = embed_model(vocab_size=len(char2int), input_length=max_root, 
                                      decoder_input=max_word, dec_output_length=len(char2int),
                                      n_feature=n_feature, hidden_size=hidden_size, 
                                      feat_units=feat_embed_size, embed_size=char_embed_size)
model.compile(optimizer='adam', loss='categorical_crossentropy')


In [10]:
history = model.fit_generator(gen, steps_per_epoch=n_batches, epochs = 50)

W0728 22:38:45.557881 10284 deprecation.py:323] From C:\Users\amany\.conda\envs\tf2\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/50
98/98 [==============================] - 16s 165ms/step - loss: 1.5945
Epoch 2/50
98/98 [==============================] - 12s 122ms/step - loss: 1.2743
Epoch 3/50
98/98 [==============================] - 11s 112ms/step - loss: 1.2161
Epoch 4/50
98/98 [==============================] - 11s 112ms/step - loss: 1.1502
Epoch 5/50
98/98 [==============================] - 11s 113ms/step - loss: 0.9976
Epoch 6/50
98/98 [==============================] - 11s 113ms/step - loss: 0.8067
Epoch 7/50
98/98 [==============================] - 11s 113ms/step - loss: 0.6425
Epoch 8/50
98/98 [==============================] - 11s 113ms/step - loss: 0.5038
Epoch 9/50
98/98 [==============================] - 11s 113ms/step - loss: 0.3659
Epoch 10/50
98/98 [==============================] - 11s 113ms/step - loss: 0.2691
Epoch 11/50
98/98 [==============================] - 11s 114ms/step - loss: 0.1962
Epoch 12/50
98/98 [==============================] - 11s 114ms/step - loss: 0.1465
Epoch 13/50
9

In [11]:
test_data = pre.convert(char2int, feat2val, max_r, max_w, train_set=False)
test_gen = pre.gen(test_data, batch_size)


In [12]:
def predict(infenc, infdec, inputs, n_steps):
    # encode
    state = infenc.predict(inputs)
#     start of sequence input
    state = [state[0], state[1], state[0], state[1]]
    target_seq = np.array([char2int['<']]*len(inputs[0])).reshape((len(inputs[0]), 1))
#     start[0] = 1
#     target_seq = np.array(start).reshape(1, 1, cardinality)
    # collect predictions
    outputs = list()
    for t in range(n_steps):
        # predict next char
        yhat, h1, c1, h2, c2 = infdec.predict([target_seq] + state)
        # store prediction
#         output.append(yhat[0,0,:])
        # update state
        state = [h1, c1, h2, c2]
#         print(yhat.shape)
        # update target sequence
        target_seq = yhat.argmax(axis=2)
#         print(target_seq.shape)
        outputs.append(target_seq)
    return np.stack(outputs)

In [13]:
# shows sample examples and calculates accuracy
test_batches = len(test_data[0]) // batch_size
total, correct = 0, 0
in_word = 0
sims = []
for b in range(test_batches - 1):
    # get data from test data generator
    [root, feat, dec_in], y = next(test_gen)
    pred = predict(encoder, decoder, [root, feat], max_word)
    for k in range(batch_size):
        indexes = pred[:, k].flatten()
        r = ''.join(pre.index_to_word(root[k], int2char)).strip()[1:-1]
        w = ''.join(pre.index_to_word(dec_in[k], int2char)).strip()[1:-1]
        t = ''.join(pre.index_to_word(indexes, int2char)).strip()[:-1]
        if w == t:
            correct += 1
        

    total += batch_size
    

print('Exact Accuracy: %.2f%%' % (float(correct)/float(total)*100.0))

Exact Accuracy: 0.12%
